# Set Up

In [ ]:
!pip install db-sqlite3

In [ ]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("sqlite.db")

# Creating and Populating Tables

In [ ]:
conn.execute("DROP TABLE IF EXISTS lab7_rooms")
try:
    conn.execute("""
CREATE TABLE IF NOT EXISTS lab7_rooms (
  RoomCode char(5) PRIMARY KEY,
  RoomName varchar(30) NOT NULL,
  Beds int(11) NOT NULL,
  bedType varchar(8) NOT NULL,
  maxOcc int(11) NOT NULL,
  basePrice DECIMAL(6,2) NOT NULL,
  decor varchar(20) NOT NULL,
  UNIQUE (RoomName)
)
  """)

    conn.execute("""
INSERT INTO lab7_rooms (RoomCode, RoomName, Beds, bedType, maxOcc, basePrice, decor) VALUES
  ('AOB', 'Abscond or bolster', 2, 'Queen', 4, 175, 'traditional'),
  ('CAS', 'Convoke and sanguine', 2, 'King', 4, 175, 'traditional'),
  ('FNA', 'Frugal not apropos', 2, 'King', 4, 250, 'traditional'),
  ('HBB', 'Harbinger but bequest', 1, 'Queen', 2, 100, 'modern'),
  ('IBD', 'Immutable before decorum', 2, 'Queen', 4, 150, 'rustic'),
  ('IBS', 'Interim but salutary', 1, 'King', 2, 150, 'traditional'),
  ('MWC', 'Mendicant with cryptic', 2, 'Double', 4, 125, 'modern'),
  ('RND', 'Recluse and defiance', 1, 'King', 2, 150, 'modern'),
  ('RTE', 'Riddle to exculpate', 2, 'Queen', 4, 175, 'rustic'),
  ('TAA', 'Thrift and accolade', 1, 'Double', 2, 75, 'modern')
  """)

    conn.commit()
except Exception as e:
    print(e)

In [ ]:
conn.execute("DROP TABLE IF EXISTS lab7_reservations")
try:
    conn.execute("""
CREATE TABLE IF NOT EXISTS lab7_reservations (
  CODE int(11) PRIMARY KEY,
  Room char(5) NOT NULL,
  CheckIn date NOT NULL,
  Checkout date NOT NULL,
  Rate DECIMAL(6,2) NOT NULL,
  LastName varchar(15) NOT NULL,
  FirstName varchar(15) NOT NULL,
  Adults int(11) NOT NULL,
  Kids int(11) NOT NULL,
  FOREIGN KEY (Room) REFERENCES lab7_rooms (RoomCode)
)
  """)

    conn.execute("""
INSERT INTO lab7_reservations (CODE, Room, CheckIn, CheckOut, Rate, LastName, FirstName, Adults, Kids) VALUES
  ('10105', 'HBB', '2023-12-13', '2023-12-25', 100, 'SELBIG', 'CONRAD', 1, 0),
  ('10183', 'IBD', '2023-09-19', '2023-09-20', 150, 'GABLER', 'DOLLIE', 2, 0),
  ('10449', 'RND', '2023-09-30', '2023-10-01', 150, 'KLESS', 'NELSON', 1, 0),
  ('10489', 'RND', '2023-02-02', '2023-02-05', 218.75, 'CARISTO', 'MARKITA', 2, 1),
  ('10500', 'MWC', '2023-12-11', '2023-12-13', 90, 'YESSIOS', 'ANNIS', 1, 0),
  ('10501', 'MWC', '2023-12-13', '2023-12-17', 90, 'YESSIOS', 'ANNIS', 1, 0),
  ('10503', 'HBB', '2023-12-18', '2023-12-20', 90, 'YESSIOS', 'ANNIS', 1, 0),
  ('10574', 'FNA', '2023-11-26', '2023-12-03', 287.5, 'SWEAZY', 'ROY', 2, 1),
  ('10984', 'RTE', '2023-12-09', '2023-12-13', 201.25, 'ZULLO', 'WILLY', 2, 1),
  ('10990', 'TAA', '2023-12-05', '2023-12-15', 175, 'TRACHSEL', 'DAMIEN', 1, 3),
  ('11631', 'AOB', '2023-12-20', '2023-12-24', 312.5, 'ESPINO', 'MARCELINA', 2, 1),
  ('11645', 'MWC', '2023-05-13', '2023-05-19', 135, 'SWAIT', 'DAN', 2, 1),
  ('11703', 'MWC', '2023-09-10', '2023-09-11', 172.5, 'HAVIS', 'SHERILYN', 2, 0),
  ('11718', 'CAS', '2023-12-18', '2023-12-19', 157.5, 'GLIWSKI', 'DAN', 2, 1),
  ('11857', 'IBD', '2023-10-27', '2023-10-29', 187.5, 'HARDINA', 'LORITA', 4, 0),
  ('11996', 'IBS', '2023-09-14', '2023-09-16', 187.5, 'BURBANK', 'ROBERT', 1, 0),
  ('12085', 'IBD', '2023-09-04', '2023-09-08', 135, 'GLASGLOW', 'EMMANUEL', 2, 0),
  ('12138', 'AOB', '2023-12-12', '2023-12-15', 150, 'SHARIAT', 'JARRED', 1, 0),
  ('12142', 'IBS', '2023-12-13', '2023-12-23', 175, 'JUNOR', 'LENNY', 3, 1),
  ('12258', 'RTE', '2022-04-23', '2022-04-27', 175, 'KANNEL', 'RODGER', 1, 0),
  ('12631', 'CAS', '2023-12-12', '2023-12-14', 175, 'ONEEL', 'PASQUALE', 1, 0),
  ('12686', 'AOB', '2021-01-12', '2021-01-15', 85, 'GROWNEY', 'MELVIN', 2, 0),
  ('12689', 'AOB', '2023-12-15', '2023-12-20', 85, 'GROWNEY', 'MELVIN', 2, 0)
  """)

    conn.commit()
except Exception as e:
    print(e)

# FR1: Rooms and Rates

In [ ]:
try:
    cursor = conn.cursor()
    cursor.execute("""
SELECT
  ro.RoomName,
  re.PopularityScore,
  COALESCE(subre2.NextAvailableDate, CURRENT_DATE),
  subre.CheckOut,
  JULIANDAY(subre.CheckOut) - JULIANDAY(subre.CheckIn)
FROM lab7_rooms AS ro
  LEFT JOIN (
    SELECT
      Room,
      ROUND(COUNT(CASE WHEN CheckOut >= DATE(CURRENT_DATE, '-180 day') THEN 1 END)/180.0, 2) AS PopularityScore
    FROM lab7_reservations
    GROUP BY Room
  ) AS re ON ro.RoomCode = re.Room
  LEFT JOIN (
    SELECT Room, CheckIn, CheckOut,
      ROW_NUMBER() OVER (PARTITION BY Room ORDER BY CheckOut DESC) AS row_num
    FROM lab7_reservations
    WHERE CheckOut < CURRENT_DATE
  ) AS subre ON subre.row_num = 1 AND subre.Room = ro.RoomCode
  LEFT JOIN (
    SELECT Room, COALESCE(MAX(nextco), MAX(CheckOut)) AS NextAvailableDate
    FROM (
      SELECT Room, CheckIn, CheckOut,
        LAG(CheckIn) OVER (PARTITION BY Room ORDER BY CheckOut DESC) AS nextci,
        LAG(CheckOut) OVER (PARTITION BY Room ORDER BY CheckOut DESC) AS nextco
      FROM lab7_reservations
    ) AS l
    WHERE Checkout >= CURRENT_DATE AND (JULIANDAY(nextci) - JULIANDAY(CheckOut) = 0 OR JULIANDAY(nextci) - JULIANDAY(CheckOut) IS NULL)
    GROUP BY Room
  ) AS subre2 ON subre2.Room = ro.RoomCode
""")
    df = pd.DataFrame(cursor.fetchall(), columns = ['Room Name', 'Popularity Score', 'Next Available Date', 'Most Recent Stay: Check Out Date', 'Most Recent Stay: Duration in Days'])
finally:
    cursor.close()

df

,Room Name,Popularity Score,Next Available Date,Most Recent Stay: Check Out Date,Most Recent Stay: Duration in Days
0,Abscond or bolster,0.02,2023-12-24,2021-01-15,3.0
1,Convoke and sanguine,0.01,2023-12-19,2023-12-14,2.0
2,Frugal not apropos,0.01,2023-12-15,2023-12-03,7.0
3,Harbinger but bequest,0.01,2023-12-25,None,NaN
4,Immutable before decorum,0.02,2023-12-15,2023-10-29,2.0
5,Interim but salutary,0.01,2023-12-23,2023-09-16,2.0
6,Mendicant with cryptic,0.02,2023-12-17,2023-12-13,2.0
7,Recluse and defiance,0.01,2023-12-15,2023-10-01,1.0
8,Riddle to exculpate,0.01,2023-12-15,2023-12-13,4.0
9,Thrift and accolade,0.01,2023-12-15,None,NaN


# FR2: Reservations

In [ ]:
from datetime import datetime, timedelta

fn = input("First Name: ")
ln = input("Last Name: ")
rc = input("Desired Room Code (Type 'Any' for no preference): ")
bt = input("Desired Bed Type (Type 'Any' for no preference): ")
ci = input("Check in date: ")
co = input("Check out date: ")
nc = input("Number of Children: ")
na = input("Number of Adults: ")

q = """
    SELECT *
    FROM lab7_rooms
    WHERE (RoomCode IN (
      SELECT DISTINCT Room
      FROM (
        SELECT *,
          COALESCE(LAG(CheckIn) OVER (PARTITION BY Room ORDER BY CheckOut DESC), '2099-12-31') AS NextCheckIn
        FROM lab7_reservations
      ) AS c
      WHERE (? >= CheckOut AND ? <= NextCheckIn)
    )
    OR RoomCode NOT IN (
      SELECT DISTINCT Room
      FROM lab7_reservations
    ))
    AND maxOcc >= ?
"""

paramsTuple = (ci, co, str(int(nc) + int(na)))

if bt != "Any":
  q += " AND bedType = ?"
  paramsTuple += (bt,)
if rc != "Any":
  q += " AND RoomCode = ?"
  paramsTuple += (rc,)

try:
    cursor = conn.cursor()
    cursor.execute("SELECT MAX(maxOcc) FROM lab7_rooms")

    start = datetime.strptime(ci, '%Y-%m-%d')
    end = datetime.strptime(co, '%Y-%m-%d')

    if int(nc) + int(na) > cursor.fetchall()[0][0]:
      print("No suitable rooms available")
    elif start > end:
      print("Start date must come before end date")
    else:
      cursor.execute(q, paramsTuple)
      res = cursor.fetchall()
      if len(res) != 0:
        print(res)
        if len(res) == 1:
          sel = input("Does this room look good? Type 'Y' or 'Cancel':")
          if sel == 'Y':
            sel = '0'
        else:
          sel = input("Type row number (starting with 0) OR Type 'Cancel' if you would like to start over: ")
      else:

        if len(paramsTuple) > 3:

          q = """
            SELECT *
            FROM lab7_rooms
            WHERE (RoomCode IN (
              SELECT DISTINCT Room
              FROM (
                SELECT *,
                  COALESCE(LAG(CheckIn) OVER (PARTITION BY Room ORDER BY CheckOut DESC), '2099-12-31') AS NextCheckIn
                FROM lab7_reservations
              ) AS c
              WHERE ? >= CheckOut AND ? <= NextCheckIn
            )
            OR RoomCode NOT IN (
              SELECT DISTINCT Room
              FROM lab7_reservations
            ))
            AND maxOcc >= ? ORDER BY
          """
          if bt != "Any":
            q += " CASE WHEN bedType = ? THEN 0 ELSE 1 END,"
          if rc != "Any":
            q += " CASE WHEN decor = (SELECT decor FROM lab7_rooms WHERE RoomCode = ?) THEN 0 ELSE 1 END,"
          q += " RoomCode"

          cursor.execute(q, paramsTuple)
          res = cursor.fetchall()

        if len(res) == 0:
          print("Couldn't find any similar matches, please try again")
          raise SystemExit("Program Quit")

        res = res[:5]
        print(res)
        sel = input("Couldn't find exact match. Closest results given. Type row number (starting with 0) OR Type 'Cancel' if you would like to start over: ")

      if sel == "Cancel":
        print("Thank you, come again")
        raise SystemExit("Program Quit")

      room = res[int(sel)]

      cursor.execute("SELECT MAX(CODE) FROM lab7_reservations")
      rNumber = str(cursor.fetchall()[0][0] + 1)

      weekdays = 0
      weekends = 0
      current = start

      while current <= end:
        if current.weekday() < 5:
          weekdays += 1
        else:
          weekends += 1
        current += timedelta(days = 1)

      total = str((weekdays * room[-2]) + (weekends * room[-2] * 1.1))
      rate = str(round(float(total)/(weekdays + weekends), 2))

      com = "INSERT INTO lab7_reservations (CODE, Room, CheckIn, CheckOut, Rate, LastName, FirstName, Adults, Kids) VALUES ('" + rNumber + "', '" + room[0] + "', ?, ?, ?, ?, ?, ?, ?)"
      cursor.execute(com, (ci, co, rate, ln, fn, na, nc))

      print(fn + " " + ln + "\nRoom Code: " + room[0] + "\nRoom Name: " + room[1] + "\nBed Type: " + room[3] + "\nFROM: " + ci + "\nTO: " + co + "\nAdults: " + str(na) + "\nChildren: " + str(nc) + "\nTotal Cost: " + total)


finally:
    cursor.close()



First Name: Brandon
Last Name: Kim
Desired Room Code (Type 'Any' for no preference): Any
Desired Bed Type (Type 'Any' for no preference): Queen
Check in date: 2023-12-31
Check out date: 2024-01-06
Number of Children: 1
Number of Adults: 1
[('AOB', 'Abscond or bolster', 2, 'Queen', 4, 175, 'traditional'), ('HBB', 'Harbinger but bequest', 1, 'Queen', 2, 100, 'modern'), ('IBD', 'Immutable before decorum', 2, 'Queen', 4, 150, 'rustic'), ('RTE', 'Riddle to exculpate', 2, 'Queen', 4, 175, 'rustic')]
Type row number (starting with 0) OR Type 'Cancel' if you would like to start over: 1
Brandon Kim
Room Code: HBB
Room Name: Harbinger but bequest
Bed Type: Queen
FROM: 2023-12-31
TO: 2024-01-06
Adults: 1
Children: 1
Total Cost: 720.0


# FR3: Reservation Cancellation

In [ ]:
cancel = input("Enter your reservation code: ")
confirm = input("Are you sure you want to delete reservation with code: " + cancel + "? (Type Y/N) ")

try:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM lab7_reservations WHERE CODE =?", [cancel])
    if len(cursor.fetchall()) == 0:
      print("Error: There is no reservation with the code: " + cancel)
    elif confirm == 'Y':
      cursor.execute("DELETE FROM lab7_reservations WHERE CODE = ?", [cancel])
      print("Successfully deleted reservation with code: " + cancel)
finally:
    cursor.close()

Enter your reservation code: 10105
Are you sure you want to delete reservation with code: 10105? (Type Y/N) Y
Successfully deleted reservation with code: 10105


# FR4: Detailed Reservation Information

In [ ]:
def start(q):
  if q[-1] == "?":
    return " AND"
  else:
    return ""

fn = input("First Name: ")
ln = input("Last Name: ")
ci = input("Check in date: ")
co = input("Check out date: ")
roc = input("Room Code: ")
rec = input("Reservation Code: ")

q = "SELECT CODE, Room, CheckIn, CheckOut, Rate, LastName, FirstName, Adults, Kids, RoomName, Beds, bedType, maxOcc, basePrice, decor FROM lab7_reservations INNER JOIN lab7_rooms ON Room = RoomCode WHERE"
paramsTuple = ()

for i in range(6):
  params = [fn, ln, ci, co, roc, rec]
  calls = [" FirstName LIKE ", " LastName LIKE ", " CheckIn = ", " CheckOut = ", " Room LIKE ", " CODE LIKE "]

  if params[i] != "":
    q += start(q) + calls[i] + "?"
    paramsTuple += (params[i],)

try:
    cursor = conn.cursor()
    if len(paramsTuple) == 0:
      cursor.execute(q[:-5])
    else:
      cursor.execute(q, paramsTuple)
    result = cursor.fetchall()
    columns = ['Reservation Code', 'Room Code', 'CheckIn', 'CheckOut', 'Rate', 'Last Name', 'First Name', 'Adults', 'Kids', 'Room Name', 'Beds', 'Bed Type', 'Max Occ', 'Base Price', 'Decor']
    df = pd.DataFrame(result, columns = columns)
finally:
    cursor.close()

df

First Name: 
Last Name: 
Check in date: 
Check out date: 
Room Code: 
Reservation Code: 10449


,Reservation Code,Room Code,CheckIn,CheckOut,Rate,Last Name,First Name,Adults,Kids,Room Name,Beds,Bed Type,Max Occ,Base Price,Decor
0,10489,RND,2023-02-02,2023-02-05,218.75,CARISTO,MARKITA,2,1,Recluse and defiance,1,King,2,150,modern


#FR5: Revenue

In [ ]:
try:
    cursor = conn.cursor()
    cursor.execute("""
SELECT
  Room,

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '01' AND strftime('%m', co2) != '01' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '01' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Jan',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '02' AND strftime('%m', co2) != '02' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '02' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Feb',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '03' AND strftime('%m', co2) != '03' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '03' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Mar',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '04' AND strftime('%m', co2) != '04' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '04' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Apr',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '05' AND strftime('%m', co2) != '05' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '05' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'May',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '06' AND strftime('%m', co2) != '06' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '06' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Jun',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '07' AND strftime('%m', co2) != '07' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '07' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Jul',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '08' AND strftime('%m', co2) != '08' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '08' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Aug',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '09' AND strftime('%m', co2) != '09' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '09' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Sep',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '10' AND strftime('%m', co2) != '10' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '10' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Oct',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '11' AND strftime('%m', co2) != '11' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '11' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Nov',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '12' AND strftime('%m', co2) != '12' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '12' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Dec',

  ROUND(SUM((julianday(CheckOut) - julianday(CheckIn)) * Rate), 0) AS 'Total'
FROM (
    SELECT
        *,
        CASE WHEN strftime('%m', ci1) != strftime('%m', co2) THEN date(ci1, '+1 month', 'start of month', '-1 day') ELSE co2 END AS co1,
        CASE WHEN strftime('%m', ci1) != strftime('%m', co2) THEN date(co2, 'start of month') ELSE ci1 END AS ci2
    FROM (
        SELECT
            *,
            CASE WHEN strftime('%Y', CheckIn) != strftime('%Y', date('now')) THEN date('now', 'start of year') ELSE CheckIn END AS ci1,
            CASE WHEN strftime('%Y', CheckOut) != strftime('%Y', date('now')) THEN date('now', 'start of year', '+1 year', '-1 day') ELSE CheckOut END AS co2
        FROM lab7_reservations
        WHERE strftime('%Y', CheckIn) = strftime('%Y', date('now')) OR strftime('%Y', CheckOut) = strftime('%Y', date('now'))
    ) AS thisyear
) AS newintervals
GROUP BY Room

UNION

SELECT 'Total' AS Room, SUM(Jan) AS Jan, SUM(Feb) AS Feb, SUM(Mar) AS Mar, SUM(Apr) AS Apr, SUM(May) AS May, SUM(Jun) AS Jun, SUM(Jul) AS Jul, SUM(Aug) AS Aug, SUM(Sep) AS Sep, SUM(Oct) AS Oct, SUM(Nov) AS Nov, SUM(Dec) AS Dec,
SUM(Total) AS Total
FROM (
SELECT
  Room,

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '01' AND strftime('%m', co2) != '01' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '01' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Jan',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '02' AND strftime('%m', co2) != '02' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '02' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Feb',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '03' AND strftime('%m', co2) != '03' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '03' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Mar',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '04' AND strftime('%m', co2) != '04' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '04' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Apr',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '05' AND strftime('%m', co2) != '05' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '05' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'May',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '06' AND strftime('%m', co2) != '06' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '06' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Jun',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '07' AND strftime('%m', co2) != '07' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '07' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Jul',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '08' AND strftime('%m', co2) != '08' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '08' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Aug',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '09' AND strftime('%m', co2) != '09' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '09' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Sep',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '10' AND strftime('%m', co2) != '10' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '10' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Oct',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '11' AND strftime('%m', co2) != '11' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '11' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Nov',

  ROUND(
    SUM(CASE WHEN strftime('%m', ci1) = '12' AND strftime('%m', co2) != '12' THEN (julianday(co1) - julianday(ci1) + 1) * Rate ELSE 0 END +
        CASE WHEN strftime('%m', ci2) = '12' THEN (julianday(co2) - julianday(ci2)) * Rate ELSE 0 END), 0) AS 'Dec',

  ROUND(SUM((julianday(CheckOut) - julianday(CheckIn)) * Rate), 0) AS 'Total'
FROM (
    SELECT
        *,
        CASE WHEN strftime('%m', ci1) != strftime('%m', co2) THEN date(ci1, '+1 month', 'start of month', '-1 day') ELSE co2 END AS co1,
        CASE WHEN strftime('%m', ci1) != strftime('%m', co2) THEN date(co2, 'start of month') ELSE ci1 END AS ci2
    FROM (
        SELECT
            *,
            CASE WHEN strftime('%Y', CheckIn) != strftime('%Y', date('now')) THEN date('now', 'start of year') ELSE CheckIn END AS ci1,
            CASE WHEN strftime('%Y', CheckOut) != strftime('%Y', date('now')) THEN date('now', 'start of year', '+1 year', '-1 day') ELSE CheckOut END AS co2
        FROM lab7_reservations
        WHERE strftime('%Y', CheckIn) = strftime('%Y', date('now')) OR strftime('%Y', CheckOut) = strftime('%Y', date('now'))
    ) AS thisyear
) AS newintervals
GROUP BY Room) AS tab
    """)
    df = pd.DataFrame(cursor.fetchall(),
                      columns = ['Room Code', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Total'])
finally:
    cursor.close()

df

,Room Code,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Total
0,AOB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2125.0,2125.0
1,CAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,508.0,508.0
2,FNA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1438.0,575.0,2013.0
3,HBB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1380.0,1380.0
4,IBD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,690.0,375.0,0.0,0.0,1065.0
5,IBS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,375.0,0.0,0.0,1750.0,2125.0
6,MWC,0.0,0.0,0.0,0.0,810.0,0.0,0.0,0.0,173.0,0.0,0.0,540.0,1523.0
7,RND,0.0,656.0,0.0,0.0,0.0,0.0,0.0,0.0,150.0,0.0,0.0,0.0,806.0
8,RTE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,805.0,805.0
9,TAA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1750.0,1750.0
